In [34]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #支持多条语句一起输出

import pandas as pd
import numpy as np
import quantaxis_ext
import matplotlib.pyplot as plt
import datetime
import settings
import calculator

#使用的指数代码
zs_symbol='000300'

In [35]:
#指数日线数据
zs_daily=quantaxis_ext.fetch_index_day_adv(zs_symbol,settings.stock_daily_start,settings.stock_daily_end)

In [36]:
#指数日收益
zs_daily_return=calculator.calc_daily_return(zs_daily['close'])

In [37]:
stock_list=quantaxis_ext.fetch_stock_list()
stock_list.head(1)

,code,decimal_point,name,pre_close,sec,sse,volunit
code,,,,,,,
000001,000001,2,平安银行,456.0,stock_cn,sz,100


In [38]:
stock_list=stock_list[stock_list['sse']=='sz'][['code','name']].set_index('code')

In [39]:
stock_list.head()

,name
code,
000001,平安银行
000002,万 科Ａ
000004,国农科技
000005,世纪星源
000006,深振业Ａ


In [40]:
%%time

daily_return_dict={}
for code in stock_list.index:
    stock_daily=quantaxis_ext.fetch_stock_day_adv(code,settings.stock_daily_start,settings.stock_daily_end)
    if stock_daily.empty:
        continue
    stock_daily_return=calculator.calc_daily_return(stock_daily['close'])
    daily_return_dict[code]=daily_return=pd.DataFrame({zs_symbol:zs_daily_return,code:stock_daily_return})

QA Error QA_fetch_stock_day_adv parameter code=002945 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=002948 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300755 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300757 , start=1990-01-01, end=2018-12-31 call QA_fetch_stock_day return None
Wall time: 2min 11s


In [41]:
daily_return_dict['000002'].head()

,000300,000002
date,,
1991-12-24,NaN,0.096491
1991-12-25,NaN,-0.032000
1991-12-26,NaN,0.000000
1991-12-27,NaN,0.004132
1991-12-28,NaN,0.020576
